In [1]:
import pandas as pd  #reading the csv file.

df = pd.read_csv('customer_shopping_behavior.csv')

In [2]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [25]:
df.isnull().sum() #checks missing values, gives the total count of the values

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [26]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [27]:
df.isnull().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [7]:
# creating uniform column names to snake case

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

# changing the name of a column
df = df.rename(columns={'purchase_amount_(usd)': 'purchase_amount'})

In [8]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [9]:
# Creating a column age_group categorizing age into order, adult, middle age, senior

labels = ['Young Adult', 'Adult', 'Middle-age', 'Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels =labels)

In [10]:
df[['age','age_group']].head()

,age,age_group
0,55,Middle-age
1,19,Young Adult
2,50,Middle-age
3,21,Young Adult
4,45,Middle-age


In [11]:
#Create purchase frequency days mapping number of days to pruchase_frequency_days

frequency_mapping = {
    'Fortnightly' : 14,
    'Weekly': 7,
    "Monthly" : 30,
    'Quaterly': 90,
    'Bi-Weekly': 14,
    'Annually': 365,
    'Every 3 Months':90
}

df ['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

In [12]:
total_customers = len(df)
print(total_customers)

3900


In [13]:
df[['discount_applied','promo_code_used']].head(10)

,discount_applied,promo_code_used
0,Yes,Yes
1,Yes,Yes
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,Yes,Yes
6,Yes,Yes
7,Yes,Yes
8,Yes,Yes
9,Yes,Yes


In [14]:
#reducing redundancy 
(df['discount_applied'] == df['promo_code_used']).all() #checking if both the columns hold same information or not

df = df.drop('promo_code_used', axis = 1) #dropping the column

In [15]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_group', 'purchase_frequency_days'],
      dtype='object')

In [37]:
#TO CONNECT VS CODE TO POSTGRESQL

!pip install psycopg2-binary
!pip install sqlalchemy
%pip install psycopg[binary]


Note: you may need to restart the kernel to use updated packages.


In [38]:
#connecting to postgresql
from sqlalchemy import create_engine 
from sqlalchemy.engine import URL


url = URL.create(
drivername = 'postgresql+psycopg',
username = 'postgres',
password = 'Garad@110512',
host = 'localhost',
port = 5433,
database = 'customer_behaviour',
)


engine = create_engine(url)

#loading Dataframe into postgresql


df.to_sql(name ='customer',con=engine, if_exists='replace', index =False)

print("Data successfully loaded into  database.")

Data successfully loaded into  database.


In [24]:
subscription_status = df.groupby("subscription_status").size()




In [25]:
subscription_status

subscription_status
No     2847
Yes    1053
dtype: int64

In [23]:
import matplotlib.pyplot as plt

labels = subscription_status.index
sizes = subscription_status.values

plt.figure()
plt.pie(
    sizes,
    labels=labels,
    autopct="%1.1f%%",
    startangle=90,
    wedgeprops=dict(width=0.4)
)

plt.title("Subscription Status Distribution")
plt.show()


NameError: name 'subscription_status' is not defined